## 1. Basic API Calling

In [3]:
!pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/1e/18/98a99ad95133c6a6e2005fe89faedf294a748bd5dc803008059409ac9b1e/python_dotenv-1.1.0-py3-none-any.whl.metadata
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [5]:
load_dotenv()
print(os.getenv("OPENAI_API_KEY"))

sk-proj-xfpvHlO8T4JsUKTtvZu2T3BlbkFJtW8Ph4Nf5KgapO6O6f2e


In [6]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "Write a limerick about the Python programming language.",
        },
    ],
)

response = completion.choices[0].message.content
print(response)

In a world full of code so diverse,  
Python stands out, quite terse.  
With indentations neat,  
And functions so sweet,  
Coding feels more like a verse.


## 2. Structured Output

In [1]:
!pip install pydantic


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pydantic import BaseModel

In [9]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 1: Define the response format in a Pydantic model

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# Step 2: Call the model

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

# Step 3: Parse the response

event = completion.choices[0].message.parsed

In [13]:
print("Event :", event.name)
print("Date :", event.date)
print("Participants :", event.participants)

Event : Science Fair
Date : Friday
Participants : ['Alice', 'Bob']


In [17]:
event = completion.choices[0].message
event

ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']))

In [18]:
event = completion.choices[0].message.parsed
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

## 3. Tool-Use Design Pattern

In [19]:
import json
import requests
from pydantic import BaseModel, Field

In [20]:
# Step 1: Define Tools

def get_weather(latitude, longitude):
    """Public API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

test_tool = get_weather(6.9271,79.8612)
test_tool

{'time': '2025-04-14T04:30',
 'interval': 900,
 'temperature_2m': 30.7,
 'wind_speed_10m': 6.2}